In [ ]:
import pandas as pd
import json
import re

c:\Users\Usuario\Desktop\UNI\Cuarto\TFG\Pruebas_Polaridad\Generalistas\fine-tuning


In [61]:
output_normal = "batch output/BatchOpenAI_GPT-4o-mini_2020_Son500_batch_67ffc9ae8e408190af8b9211a9ab9d49_output.jsonl"
output_finetuning = "batch output/BatchOpenAI_GPT-4o-mini_2020_Son500_finetuning_batch_67ffca051e1081909e493e205450fa52_output.jsonl"
output_finetuning_v2 = "batch output/BatchOpenAI_GPT-4o-mini_2020_Son500_finetuning_v2_batch_68178b1335e88190bcf4ed4be9f0fd6c_output.jsonl"
outoput =[output_normal, output_finetuning, output_finetuning_v2]

In [62]:
def extract_contents_from_jsonl(path):
    contents = []
    ids = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line)
            try:
                content = data["response"]["body"]["choices"][0]["message"]["content"]
                id = data["custom_id"]
                id = re.sub(r"^request-", "", id)
                contents.append(content)
                ids.append(id)
                
            except KeyError:
                contents.append("")
                ids.append("")
    return contents, ids

In [63]:
normal_polarity, normal_id = extract_contents_from_jsonl(output_normal)
finetuning_polarity, finetuning_id = extract_contents_from_jsonl(output_finetuning)
finetuning_polarity2, finetuning_id2 = extract_contents_from_jsonl(output_finetuning_v2)

In [64]:
df_normal = pd.DataFrame({"Id": normal_id, "Polarity_GPT-4o-mini": normal_polarity})
df_finetuning = pd.DataFrame({"Id": finetuning_id, "Polarity_GPT-4o-mini_Finetuning": finetuning_polarity})
df_finetuning2 = pd.DataFrame({"Id": finetuning_id2, "Polarity_GPT-4o-mini_Finetuning_v2": finetuning_polarity2})

In [65]:
merged_df = pd.merge(df_normal, df_finetuning, on="Id")
merged_df = merged_df.sort_values(by="Id")

merged_df2 = pd.merge(merged_df, df_finetuning2, on="Id")
merged_df2 = merged_df2.sort_values(by="Id")

In [ ]:
df = pd.read_excel('../../Spain_test_2020.xlsx')

df["Polarity_GPT-4o-mini"] = merged_df2["Polarity_GPT-4o-mini"]
df["Polarity_GPT-4o-mini_Finetuning"] = merged_df2["Polarity_GPT-4o-mini_Finetuning"]
df["Polarity_GPT-4o-mini_Finetuning_v2"] = merged_df2["Polarity_GPT-4o-mini_Finetuning_v2"]

df.to_excel('Clasificación_GPT-4o-mini_Fine-tuning.xlsx', index=False)
